# Complete DAX 40 Comprehensive Portfolio Analysis

In [1]:
# %pip install yfinance pandas numpy matplotlib pyplot scipy.optimize seaborn requests StringIO zipfile

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import requests
from io import StringIO
import seaborn as sns
import statsmodels.api as sm

# Set random seed for reproducibility
np.random.seed(42)

# DAX 40 companies information
dax40_info = {
    'Adidas': {'sector': 'Apparel', 'ticker': 'ADS.DE'},
    'Airbus': {'sector': 'Aerospace & Defence', 'ticker': 'AIR.DE'},
    'Allianz': {'sector': 'Financial Services', 'ticker': 'ALV.DE'},
    'BASF': {'sector': 'Chemicals', 'ticker': 'BAS.DE'},
    'Bayer': {'sector': 'Pharmaceuticals', 'ticker': 'BAYN.DE'},
    'Beiersdorf': {'sector': 'Consumer goods', 'ticker': 'BEI.DE'},
    'BMW': {'sector': 'Automotive', 'ticker': 'BMW.DE'},
    'Brenntag': {'sector': 'Distribution', 'ticker': 'BNR.DE'},
    'Commerzbank': {'sector': 'Financial Services', 'ticker': 'CBK.DE'},
    'Continental': {'sector': 'Automotive', 'ticker': 'CON.DE'},
    'Covestro': {'sector': 'Chemicals', 'ticker': '1COV.DE'},
    'Daimler Truck': {'sector': 'Automotive', 'ticker': 'DTG.DE'},
    'Deutsche Bank': {'sector': 'Financial Services', 'ticker': 'DBK.DE'},
    'Deutsche Börse': {'sector': 'Financial Services', 'ticker': 'DB1.DE'},
    'Deutsche Post': {'sector': 'Logistics', 'ticker': 'DHL.DE'},
    'Deutsche Telekom': {'sector': 'Telecommunication', 'ticker': 'DTE.DE'},
    'E.ON': {'sector': 'Utilities', 'ticker': 'EOAN.DE'},
    'Fresenius': {'sector': 'Healthcare', 'ticker': 'FRE.DE'},
    'Hannover Re': {'sector': 'Insurance', 'ticker': 'HNR1.DE'},
    'Heidelberg Materials': {'sector': 'Construction Materials', 'ticker': 'HEI.DE'},
    'Henkel': {'sector': 'Consumer Goods', 'ticker': 'HEN3.DE'},
    'Infineon Technologies': {'sector': 'Technology', 'ticker': 'IFX.DE'},
    'Mercedes-Benz Group': {'sector': 'Automotive', 'ticker': 'MBG.DE'},
    'Merck': {'sector': 'Pharmaceuticals', 'ticker': 'MRK.DE'},
    'MTU Aero Engines': {'sector': 'Aerospace & Defence', 'ticker': 'MTX.DE'},
    'Munich Re': {'sector': 'Financial Services', 'ticker': 'MUV2.DE'},
    'Porsche': {'sector': 'Automotive', 'ticker': 'P911.DE'},
    'Porsche SE': {'sector': 'Automotive', 'ticker': 'PAH3.DE'},
    'Qiagen': {'sector': 'Biotech', 'ticker': 'QIA.DE'},
    'Rheinmetall': {'sector': 'Aerospace & Defence', 'ticker': 'RHM.DE'},
    'RWE': {'sector': 'Utilities', 'ticker': 'RWE.DE'},
    'SAP': {'sector': 'Technology', 'ticker': 'SAP.DE'},
    'Sartorius': {'sector': 'Medical Technology', 'ticker': 'SRT3.DE'},
    'Siemens': {'sector': 'Industrials', 'ticker': 'SIE.DE'},
    'Siemens Energy': {'sector': 'Energy technology', 'ticker': 'ENR.DE'},
    'Siemens Healthineers': {'sector': 'Medical Equipment', 'ticker': 'SHL.DE'},
    'Symrise': {'sector': 'Chemicals', 'ticker': 'SY1.DE'},
    'Volkswagen Group': {'sector': 'Automotive', 'ticker': 'VOW3.DE'},
    'Vonovia': {'sector': 'Real Estate', 'ticker': 'VNA.DE'},
    'Zalando': {'sector': 'E-Commerce', 'ticker': 'ZAL.DE'}
}

# Extract tickers and create a mapping of tickers to sectors
dax40_tickers = [company['ticker'] for company in dax40_info.values()]
ticker_to_sector = {company['ticker']: company['sector'] for company in dax40_info.values()}

def load_dax40_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date)
    return data

# Load data
start_date = '2023-01-01'
end_date = '2023-12-31'
dax40_data = load_dax40_data(dax40_tickers, start_date, end_date)

# Calculate returns
returns = dax40_data['Adj Close'].pct_change().dropna()

## 1. Sector-Based Analysis

In [ ]:
# Group returns by sector
sector_returns = returns.groupby(ticker_to_sector, axis=1).mean()

print("Sector-based returns calculated.")
print(f"Shape of sector returns data: {sector_returns.shape}")
print("\nUnique sectors:")
print(sector_returns.columns.tolist())

# Visualize sector performance
plt.figure(figsize=(12, 6))
(1 + sector_returns).cumprod().plot()
plt.title('Cumulative Returns by Sector')
plt.ylabel('Cumulative Return')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small')
plt.tight_layout()
plt.show()

## 2. Fama-French FIve-Factor Model

In [ ]:
import pandas as pd
import requests
from io import BytesIO, StringIO
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import numpy as np

def get_french_data(url):
    response = requests.get(url)
    with zipfile.ZipFile(BytesIO(response.content)) as zip_file:
        file_name = zip_file.namelist()[0]
        with zip_file.open(file_name) as file:
            content = file.read().decode('utf-8')

    lines = content.split('\n')
    data_lines = []
    start_processing = False
    for line in lines:
        if not start_processing:
            if any(col in line for col in ['Date', 'Mkt-RF', 'SMB', 'HML', 'RF', 'Mom']):
                start_processing = True
                data_lines.append(line)
        elif start_processing:
            if line.strip() == '':
                break
            if not line.startswith('Copyright'):
                data_lines.append(line)

    df = pd.read_csv(StringIO('\n'.join(data_lines)), index_col=0, na_values=['-99.99', '-999'])
    
    df.index = pd.to_datetime(df.index, format='%Y%m%d')
    df = df.apply(pd.to_numeric, errors='coerce')
    df = df.div(100)
    df.columns = df.columns.str.strip()

    return df

def calculate_factor_exposures(returns, factors):
    exposures = {}
    for stock in returns.columns:
        model = sm.OLS(returns[stock], sm.add_constant(factors)).fit()
        exposures[stock] = model.params
    exposures_df = pd.DataFrame(exposures).T
    return exposures_df

# URLs for Fama-French factors
ff_url = "https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip"
momentum_url = "https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Momentum_Factor_daily_CSV.zip"

# Get Fama-French and momentum data
ff_factors = get_french_data(ff_url)
momentum_factor = get_french_data(momentum_url)

# Merge factors data
factors = ff_factors.join(momentum_factor['Mom'], how='inner')

# Filter factors data to match the date range of stock returns
start_date = returns.index.min()
end_date = returns.index.max()
factors = factors[(factors.index >= start_date) & (factors.index <= end_date)]

# Ensure the returns and factors align on the same dates
common_dates = returns.index.intersection(factors.index)
returns = returns.loc[common_dates]
factors = factors.loc[common_dates]

try:
    factor_exposures = calculate_factor_exposures(returns, factors)
    print("Factor exposures:")
    print(factor_exposures.head())

    # Convert factor exposures to float type
    factor_exposures = factor_exposures.astype(float)

    # Clip extreme values for better visualization
    lower_bound, upper_bound = np.percentile(factor_exposures.values, [1, 99])
    factor_exposures_clipped = factor_exposures.clip(lower_bound, upper_bound)

    plt.figure(figsize=(16, 12))
    sns.heatmap(factor_exposures_clipped, annot=True, cmap='coolwarm', center=0, fmt='.2f')
    plt.title('Factor Exposures Heatmap')
    plt.tight_layout()
    plt.show()

    # Print summary statistics
    print("\nFactor Exposures Summary Statistics:")
    print(factor_exposures.describe())

    # Print extreme values
    print("\nStocks with Extreme Factor Exposures:")
    for column in factor_exposures.columns:
        max_stock = factor_exposures[column].idxmax()
        min_stock = factor_exposures[column].idxmin()
        print(f"\n{column}:")
        print(f"  Max: {max_stock} ({factor_exposures.loc[max_stock, column]:.2f})")
        print(f"  Min: {min_stock} ({factor_exposures.loc[min_stock, column]:.2f})")

except Exception as e:
    print(f"An error occurred: {e}")
    print("Unable to process or visualize Fama-French data. Please check the data and format.")


## 3. Portfolio Construction Techniques

In [ ]:
def equal_weight_portfolio(returns):
    n = returns.shape[1]
    weights = np.ones(n) / n
    return weights

def risk_parity_portfolio(returns, risk_target=0.1, max_iter=1000, tolerance=1e-6):
    n = returns.shape[1]
    weights = np.ones(n) / n
    cov_matrix = returns.cov().values
    
    for _ in range(max_iter):
        risk_contributions = weights * (cov_matrix.dot(weights))
        new_weights = weights * risk_target / risk_contributions
        new_weights /= np.sum(new_weights)
        
        if np.sum(np.abs(new_weights - weights)) < tolerance:
            break
        
        weights = new_weights
    
    return weights

def min_variance_portfolio(returns):
    cov_matrix = returns.cov().values
    n = returns.shape[1]
    
    def objective(weights):
        return weights.T.dot(cov_matrix).dot(weights)
    
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0, 1) for _ in range(n))
    
    result = minimize(objective, np.ones(n) / n, method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

def mean_variance_portfolio(returns, risk_aversion=2):
    mean_returns = returns.mean().values
    cov_matrix = returns.cov().values
    n = returns.shape[1]
    
    def objective(weights):
        portfolio_return = np.dot(weights, mean_returns)
        portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
        return -portfolio_return + risk_aversion * portfolio_variance
    
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0, 1) for _ in range(n))
    
    result = minimize(objective, np.ones(n) / n, method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

# Calculate portfolio weights
equal_weight = equal_weight_portfolio(returns)
risk_parity = risk_parity_portfolio(returns)
min_var = min_variance_portfolio(returns)
mean_var = mean_variance_portfolio(returns)

portfolios = {
    'Equal Weight': equal_weight,
    'Risk Parity': risk_parity,
    'Minimum Variance': min_var,
    'Mean-Variance': mean_var
}

# Visualize portfolio weights
plt.figure(figsize=(15, 10))
for name, weights in portfolios.items():
    plt.bar(range(len(weights)), weights, alpha=0.5, label=name)
plt.xlabel('Stocks')
plt.ylabel('Weight')
plt.title('Portfolio Weights Comparison')
plt.legend()
plt.xticks(range(len(dax40_tickers)), dax40_tickers, rotation=90)
plt.tight_layout()
plt.show()

## 4. Litterman's Approach for Expected Returns

In [ ]:
def black_litterman_model(returns, market_caps, risk_aversion=2.5, tau=0.05):
    # Calculate historical returns and covariance
    hist_returns = returns.mean()
    cov_matrix = returns.cov()
    
    # Calculate market-implied equilibrium returns
    market_weights = market_caps / market_caps.sum()
    implied_returns = risk_aversion * cov_matrix.dot(market_weights)
    
    # Combine views with prior
    posterior_cov = np.linalg.inv(np.linalg.inv(tau * cov_matrix) + np.linalg.inv(cov_matrix))
    posterior_returns = posterior_cov.dot(np.linalg.inv(tau * cov_matrix).dot(implied_returns) + np.linalg.inv(cov_matrix).dot(hist_returns))
    
    return pd.Series(posterior_returns, index=returns.columns)

# Get market cap data (this is simplified, you might want to use actual market cap data)
market_caps = dax40_data['Adj Close'].iloc[-1] * dax40_data['Volume'].iloc[-1]

expected_returns = black_litterman_model(returns, market_caps)

print("Expected returns using Litterman's approach:")
print(expected_returns)

# Visualize expected returns
plt.figure(figsize=(12, 6))
expected_returns.sort_values().plot(kind='bar')
plt.title("Expected Returns (Litterman's Approach)")
plt.xlabel('Stocks')
plt.ylabel('Expected Return')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

## 5. Monte Carlo Simulations

In [ ]:
def monte_carlo_simulation(returns, weights, num_simulations=10000, time_horizon=252):
    mean_returns = returns.mean()
    cov_matrix = returns.cov()
    
    simulated_returns = np.random.multivariate_normal(mean_returns, cov_matrix, (num_simulations, time_horizon))
    portfolio_returns = np.dot(simulated_returns, weights)
    cumulative_returns = np.cumprod(1 + portfolio_returns, axis=1)
    
    return cumulative_returns

# Run simulations for each portfolio
simulations = {name: monte_carlo_simulation(returns, weights) for name, weights in portfolios.items()}

# Visualize simulations
fig, axes = plt.subplots(2, 2, figsize=(20, 15))
fig.suptitle('Monte Carlo Simulations of Portfolio Performance', fontsize=16)

for (name, sim), ax in zip(simulations.items(), axes.ravel()):
    for i in range(100):  # Plot 100 random simulations
        ax.plot(sim[i], alpha=0.1, color='blue')
    ax.plot(sim.mean(axis=0), color='red', linewidth=2)
    ax.set_title(name)
    ax.set_xlabel('Trading Days')
    ax.set_ylabel('Cumulative Returns')

plt.tight_layout()
plt.show()

## 6. Performance Evaluation and Comparison with DAX Index

In [ ]:
def calculate_portfolio_returns(weights, returns):
    return returns.dot(weights)

def calculate_sharpe_ratio(returns, risk_free_rate=0):
    excess_returns = returns - risk_free_rate
    return np.sqrt(252) * excess_returns.mean() / excess_returns.std()

# Load DAX index data
dax_index = yf.download('^GDAXI', start=start_date, end=end_date)['Adj Close']
dax_returns = dax_index.pct_change().dropna()

# Calculate cumulative returns for each portfolio and the DAX index
cumulative_returns = pd.DataFrame({name: (1 + calculate_portfolio_returns(weights, returns)).cumprod()
                                   for name, weights in portfolios.items()})
cumulative_returns['DAX Index'] = (1 + dax_returns).cumprod()

# Visualize performance comparison
plt.figure(figsize=(12, 6))
cumulative_returns.plot()
plt.title('Portfolio Performance Comparison with DAX Index')
plt.ylabel('Cumulative Return')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Calculate Sharpe ratios
sharpe_ratios = {}
for name, weights in portfolios.items():
    portfolio_returns = calculate_portfolio_returns(weights, returns)
    sharpe_ratios[name] = calculate_sharpe_ratio(portfolio_returns)
sharpe_ratios['DAX Index'] = calculate_sharpe_ratio(dax_returns)

print("Sharpe Ratios:")
for name, ratio in sharpe_ratios.items():
    print(f"{name}: {ratio:.4f}")

## 7. Time-Specific Sharpe Ratio Calculations

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def calculate_rolling_sharpe_ratio(returns, window=63, risk_free_rate=0.015):  # Using ~3 months window
    excess_returns = returns - risk_free_rate
    rolling_return = excess_returns.rolling(window=window).mean()
    rolling_std = excess_returns.rolling(window=window).std()
    return np.sqrt(252) * rolling_return / rolling_std

# Calculate portfolio returns
portfolio_returns = pd.DataFrame({name: calculate_portfolio_returns(weights, returns)
                                  for name, weights in portfolios.items()})

# Add DAX returns
portfolio_returns['DAX Index'] = dax_returns

# Calculate rolling Sharpe ratios
rolling_sharpe_ratios = pd.DataFrame({name: calculate_rolling_sharpe_ratio(returns)
                                      for name, returns in portfolio_returns.items()})

# Visualize rolling Sharpe ratios
plt.figure(figsize=(16, 8))
for column in rolling_sharpe_ratios.columns:
    plt.plot(rolling_sharpe_ratios.index, rolling_sharpe_ratios[column], label=column)

plt.title('Rolling Sharpe Ratios (3-Month Window)', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Sharpe Ratio', fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Print summary statistics
print("\nRolling Sharpe Ratios Summary Statistics:")
print(rolling_sharpe_ratios.describe())

# Print average Sharpe ratios
print("\nAverage Sharpe Ratios:")
print(rolling_sharpe_ratios.mean().sort_values(ascending=False))

## 8. Sector Allocation Analysis

In [ ]:
def calculate_sector_weights(portfolio_weights, ticker_to_sector):
    sector_weights = {}
    for ticker, weight in zip(returns.columns, portfolio_weights):
        sector = ticker_to_sector[ticker]
        sector_weights[sector] = sector_weights.get(sector, 0) + weight
    return sector_weights

# Calculate sector weights for each portfolio
sector_allocations = {name: calculate_sector_weights(weights, ticker_to_sector) 
                      for name, weights in portfolios.items()}

# Visualize sector allocations
fig, axes = plt.subplots(2, 2, figsize=(20, 15))
fig.suptitle('Sector Allocations by Portfolio Strategy', fontsize=16)

for (name, allocation), ax in zip(sector_allocations.items(), axes.ravel()):
    sectors = list(allocation.keys())
    weights = list(allocation.values())
    ax.pie(weights, labels=sectors, autopct='%1.1f%%', startangle=90)
    ax.set_title(name)

plt.tight_layout()
plt.show()

## 9. Risk Analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming we have the summary dataframe from the previous code
summary = pd.DataFrame({
    'Volatility': volatilities,
    'VaR (95%)': var_values,
    'CVaR (95%)': cvar_values,
    'Sharpe Ratio': sharpe_ratios
})

# Color palette
colors = sns.color_palette("husl", n_colors=len(summary))

# 1. Bar plots for each metric
fig, axes = plt.subplots(2, 2, figsize=(16, 14))
fig.suptitle('Portfolio Risk Metrics Comparison', fontsize=16)

metrics = ['Volatility', 'VaR (95%)', 'CVaR (95%)', 'Sharpe Ratio']
for i, metric in enumerate(metrics):
    ax = axes[i // 2, i % 2]
    summary[metric].plot(kind='bar', ax=ax, color=colors)
    ax.set_title(metric)
    ax.set_ylabel('Value')
    ax.tick_params(axis='x', rotation=45)
    for j, v in enumerate(summary[metric]):
        ax.text(j, v, f'{v:.4f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# 2. Risk-Return Scatter Plot
plt.figure(figsize=(12, 8))
for i, (index, row) in enumerate(summary.iterrows()):
    plt.scatter(row['Volatility'], row['Sharpe Ratio'], s=100, color=colors[i], label=index)
    plt.annotate(index, (row['Volatility'], row['Sharpe Ratio']), xytext=(5, 5), 
                 textcoords='offset points', fontsize=8, alpha=0.8)

plt.xlabel('Annualized Volatility')
plt.ylabel('Sharpe Ratio')
plt.title('Risk-Return Trade-off')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 3. VaR vs CVaR Comparison
plt.figure(figsize=(12, 8))
x = range(len(summary))
width = 0.35

plt.bar(x, summary['VaR (95%)'], width, label='VaR (95%)', color='skyblue', alpha=0.7)
plt.bar([i + width for i in x], summary['CVaR (95%)'], width, label='CVaR (95%)', color='salmon', alpha=0.7)

plt.xlabel('Portfolio Strategy')
plt.ylabel('Value')
plt.title('VaR vs CVaR Comparison')
plt.xticks([i + width/2 for i in x], summary.index, rotation=45)
plt.legend()

for i, (var, cvar) in enumerate(zip(summary['VaR (95%)'], summary['CVaR (95%)'])):
    plt.text(i, var, f'{var:.4f}', ha='center', va='bottom')
    plt.text(i + width, cvar, f'{cvar:.4f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 10. Reallocation Analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

def calculate_turnover(old_weights, new_weights):
    return np.sum(np.abs(new_weights - old_weights)) / 2

def equal_weight_portfolio(returns):
    return np.ones(returns.shape[1]) / returns.shape[1]

def risk_parity_portfolio(returns, risk_target=1, max_iter=1000, tolerance=1e-6):
    n = returns.shape[1]
    weights = np.ones(n) / n
    cov_matrix = returns.cov().values
    
    for _ in range(max_iter):
        risk_contributions = weights * (cov_matrix.dot(weights))
        new_weights = weights * risk_target / risk_contributions
        new_weights /= np.sum(new_weights)
        
        if np.sum(np.abs(new_weights - weights)) < tolerance:
            break
        
        weights = new_weights
    
    return weights

def min_variance_portfolio(returns):
    n = returns.shape[1]
    cov_matrix = returns.cov().values
    
    def objective(weights):
        return weights.T @ cov_matrix @ weights
    
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0, 1) for _ in range(n))
    
    result = minimize(objective, np.ones(n) / n, method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

def mean_variance_portfolio(returns, risk_aversion=1):
    n = returns.shape[1]
    mean_returns = returns.mean().values
    cov_matrix = returns.cov().values
    
    def objective(weights):
        portfolio_return = weights.T @ mean_returns
        portfolio_variance = weights.T @ cov_matrix @ weights
        return -portfolio_return + risk_aversion * portfolio_variance
    
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0, 1) for _ in range(n))
    
    result = minimize(objective, np.ones(n) / n, method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

portfolio_strategies = {
    'Equal Weight': equal_weight_portfolio,
    'Risk Parity': risk_parity_portfolio,
    'Minimum Variance': min_variance_portfolio,
    'Mean-Variance': mean_variance_portfolio
}

reallocation_dates = pd.date_range(start=returns.index[0], end=returns.index[-1], freq='QE')
turnover = {name: [] for name in portfolio_strategies.keys()}

for i in range(1, len(reallocation_dates)):
    start_date = reallocation_dates[i-1]
    end_date = reallocation_dates[i]
    old_returns = returns.loc[:start_date]
    new_returns = returns.loc[:end_date]
    
    for name, strategy in portfolio_strategies.items():
        old_weights = strategy(old_returns)
        new_weights = strategy(new_returns)
        turnover[name].append(calculate_turnover(old_weights, new_weights))

plt.figure(figsize=(12, 6))
for name, turnovers in turnover.items():
    plt.plot(reallocation_dates[1:], turnovers, label=name, marker='o')

plt.title('Portfolio Turnover over Time')
plt.xlabel('Date')
plt.ylabel('Turnover')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

avg_turnover = {name: np.mean(turnovers) for name, turnovers in turnover.items()}

print("Average Turnover:")
for name, avg in avg_turnover.items():
    print(f"{name}: {avg:.4f}")

plt.figure(figsize=(10, 6))
plt.bar(avg_turnover.keys(), avg_turnover.values())
plt.title('Average Portfolio Turnover')
plt.xlabel('Portfolio Strategy')
plt.ylabel('Average Turnover')
plt.xticks(rotation=45)
for i, v in enumerate(avg_turnover.values()):
    plt.text(i, v, f'{v:.4f}', ha='center', va='bottom')
plt.tight_layout()
plt.show()

## Conclusion

This comprehensive analysis of the DAX 40 companies includes:
1. Sector-based performance analysis
2. Fama-French five-factor model implementation
3. Various portfolio construction techniques
4. Litterman's approach for expected returns
5. Monte Carlo simulations for robustness checks
6. Performance comparison with the DAX index
7. Time-specific Sharpe ratio calculations
8. Sector allocation analysis
9. Risk analysis including volatility and Value at Risk (VaR)
10. Reallocation analysis to assess portfolio turnover

These results provide deep insights into the effectiveness of different portfolio construction techniques in the German stock market, considering both factor-based and sector-based approaches. The analysis also compares the performance of these strategies to the DAX index, evaluates their risk-adjusted returns over time, and assesses their practical implications in terms of sector exposure and reallocation costs.

Key findings and observations:
- [Insert your main findings and observations here based on the results]

Areas for further research:
- Incorporate transaction costs and taxes into the portfolio optimization process
- Explore machine learning approaches for return prediction and portfolio optimization
- Investigate the impact of different economic regimes on portfolio performance
- Conduct out-of-sample testing to assess the robustness of the strategies